## Third Movement: 

In [90]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [91]:
import os
experiment_log_file = 'experiment_3.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [92]:
%run attack_graph_MIR100.ipynb

### Use ??? Distribution

In [93]:
# The key change - enhanced random_steps function
attack_rate_list = [0]   # Not used but kept for API consistency
defense_rate_list = [0]  # Not used but kept for API consistency

In [94]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    """
    Calculate probability distribution based on edge difficulties.
    
    Args:
        route: List of nodes representing an attack path
        attack_rate: Not used in this implementation but kept for API consistency
        defense_rate: Not used in this implementation but kept for API consistency
        graph: NetworkX graph containing edge probabilities
        
    Returns:
        numpy array: Probability distribution for number of steps attacker can take
    """
    # Get edge probabilities between consecutive nodes in route
    hardness = []
    for i in range(len(route)-1):
        # Get edge data between these nodes
        edges_data = graph.get_edge_data(route[i], route[i+1])
        
        # Handle both MultiDiGraph and DiGraph edge data structures
        if isinstance(edges_data, dict):
            if 'edge_probabilities' in edges_data:
                # DiGraph case - direct dictionary
                prob = float(edges_data.get('edge_probabilities', 1.0))
                hardness.append(prob)
            else:
                # MultiDiGraph case - dictionary of edge dictionaries
                probs = []
                for key in edges_data:
                    if isinstance(edges_data[key], dict):
                        prob = edges_data[key].get('edge_probabilities', 1.0)
                        probs.append(float(prob))
                    else:
                        probs.append(float(edges_data.get('edge_probabilities', 1.0)))
                hardness.append(max(probs) if probs else 1.0)
        else:
            # No edge data found
            hardness.append(1.0)
    
    # Convert to numpy array and handle missing values
    hardness = np.array(hardness)
    hardness[np.isnan(hardness)] = 1.0  # Missing probabilities treated as certainty
    
    # Calculate PDF according to the paper's formula
    # P(n steps) = (1-p_{n+1}) * product(p_1 to p_n)
    # where p_i is probability of success at step i
    
    # Calculate cumulative products
    cumprod = np.concatenate(([1.0], np.cumprod(hardness)))
    
    # Calculate (1-p) terms
    one_minus_h = np.concatenate((1 - hardness, [1.0]))
    
    # Final PDF calculation
    pdf = one_minus_h * cumprod
    
    # Normalize to ensure it sums to 1
    pdf = pdf / np.sum(pdf)
    
    return pdf

In [95]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()


Before merging targets:
Nodes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
Edges with weights:
1 -> 5: 2.1958405355640576
2 -> 9: 0.7489215526277323
2 -> 10: 0.7489215526277323
2 -> 11: 1.0644384240729572
2 -> 16: 1.0644384240729572
3 -> 6: 1.0644384240729572
3 -> 8: 0.7489215526277323
4 -> 7: 0.7489215526277323
5 -> 15: 2.1958405355640576
6 -> 8: -0.0
7 -> 10: 0.7489215526277323
7 -> 14: 0.7489215526277323
7 -> 16: 0.7489215526277323
8 -> 10: -0.0
8 -> 14: -0.0
8 -> 16: 0.7489215526277323
9 -> 14: 1.0644384240729572
10 -> 15: -0.0
11 -> 13: 1.0644384240729572
11 -> 14: 0.7489215526277323
11 -> 16: -0.0
15 -> 12: 0.7489215526277323
15 -> 13: 0.7489215526277323
15 -> 16: 0.7489215526277323

AFTER merging targets:
Node count: 14
Edge count: 22

Edge structure after merging:
Edge 1: 0, 1 weight: 1
Edge 2: 0, 2 weight: 1
Edge 3: 0, 3 weight: 1
Edge 4: 0, 4 weight: 1
Edge 5: 1, 5 weight: 2.1958405355640576
Edge 6: 2, 9 weight: 0.7489215526277323
Edge 7: 2, 10 weight: 0.74892155

In [96]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_3.log
[1] "Sat Dec 21 17:56:53 2024"

++++++++++++++++++++++++++++++++

The virtual target nodeID is 17

attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
1 0.000000e+00
10 0.000000e+00
11 2.388662e-01
15 1.976793e-01
2 0.000000e+00
3 0.000000e+00
4 0.000000e+00
5 0.000000e+00
6 0.000000e+00
7 0.000000e+00
8 5.634545e-01
9 0.000000e+00

worst case attack strategies per goal:
          1
1 0.0000000
2 0.3411805
3 0.1376046
4 0.0000000
5 0.0000000
6 0.0000000
7 0.0000000
8 0.0000000
9 0.0000000
10 0.5212148
11 0.0000000
[1] 0.085

Defender can keep attacker success below: 0.085
Attacker can guarantee success probability of: 0.085

